In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_context('notebook')
sns.set(style='darkgrid')
from pipe import Pipe

%matplotlib inline
import pylab as pl
import holoviews as hv
hv.extension('bokeh')


In [ ]:
test_log_file = '20180312_test_log.csv'
data_dir = './20180312/'
channel_mapper= dict(
    a='sig_gen',
    b='res_volt',
    c='rec_volt',
    d='sec_volt'
)
df_log = pd.read_csv(test_log_file).drop(['test_no'], axis=1)
df_log = df_log.rename(columns=dict(primary_position='pos'))
df_log.loc[:, 'file_name'] = [f'{fn}.csv' for fn in df_log.file_name]

df_log.head()

In [ ]:
def get_data(reload=False):
    if reload:
        df = None
        for (pipe_label, pos), batch in df_log.groupby(by=['sample', 'fatigue_life']):
            p = Pipe(pipe_label, batch, channel_mapper, data_dir=data_dir, n_jobs=2, harmonic=3)
            p.process()
            if df is None:
                df = p.df
            else:
                df = df.append(p.df, ignore_index=True)
        df.to_csv('results.txt', index=False)
    df = pd.read_csv('./results.txt')
    df = df.rename(columns={'pipe': 'pipe_label'})
    return df

df = get_data(reload=False)
df = df[df.pipe_label.str.startswith('cs')]
df.loc[:, 'pipe_label'] = df.pipe_label.map(lambda s: 'virgin' if s == 'cs_0' else s)
df.head(3)

In [ ]:
df.pipe_label.unique()

In [ ]:
def get_col(df, col_name):
    df = df.pivot(index='pos', columns='pipe_label', values=col_name)
    df = df.reset_index(drop=False)
    df.index.name = col_name
    df.columns.name = None
    return df

In [ ]:
def do_plot(df, field_name, pipe_labels, title=None, ylabel=None):
    xlabel = 'Position along Pipe (inches)'
    
    dfx = get_col(df, field_name)
    pl.rcParams['figure.figsize'] = (20, 6)
    sns.set_context('talk')
    
    pl.subplot(131)
    for pipe_label in pipe_labels:
        pl.plot(dfx.pos, dfx[pipe_label], '.-', label=pipe_label, )
    pl.plot(dfx.pos, dfx.virgin, '.-', label='Virgin', )
    pl.legend(loc='best')
    pl.xlabel(xlabel)
    if ylabel is not None:
        pl.title(title);
        
    pl.subplot(132)
    for pipe_label in pipe_labels:
        pl.plot(dfx.pos, dfx[pipe_label] / dfx.virgin, '.-', label=f'{pipe_label} ratio')
    pl.xlabel(xlabel)
    pl.legend(loc='best')
    pl.title(title)
    pl.subplot(133)
    for pipe_label in pipe_labels:
        pl.plot(dfx.pos, dfx[pipe_label] - dfx.virgin, '.-', label=f'{pipe_label} diff')
    pl.xlabel(xlabel)
    pl.title(title)
    pl.legend(loc='best')

pipe_labels = [
    'cs_50',
#     'cs_65',
    'cs_80',
#     'cs_90',
    'cs_100_a',
#     'cs_100_b',
]
pl.figure()
do_plot(df, 'prim_sec_amp', pipe_labels, title='Primary-Secondary Coupling', ylabel='Mutual Inductance')

pl.figure()
do_plot(df, 'prim_rec_amp', pipe_labels, title='Primary-Receiver Coupling', ylabel='Mutual Inductance')
pl.figure()
do_plot(df, 'sec_harm_db', pipe_labels, title='3rd Harmonic Power', ylabel='dB')



In [ ]:
df.head()

In [ ]:
from daq.pico import CSV
from harmonic import Harmonic
from easier import shade
from scipy import signal

In [ ]:
file_name = './20180312/20180312-0005.csv'
df = CSV(file_name=file_name, max_sample_freq=1e9, **channel_mapper).df
# filter_cols = ['res_volt', 'sec_volt', 'rec_volt']
# for col in filter_cols:
#     # 8 pol filter at .01 of nyquist
#     b, a = signal.butter(8, 0.01)
#     df.loc[:, col] = signal.filtfilt(b, a, df[col].values, padlen=150)


harmonics = [1, 3,]
h_i_prim = Harmonic(harmonics=harmonics)
h_i_prim.fit(df.t, df.res_volt)
h_i_prim = h_i_prim.derivative()

h_v_sec = Harmonic(harmonics=harmonics)
h_v_sec.fit(df.t, df.sec_volt)

h_v_rec = Harmonic(harmonics=harmonics)
h_v_rec.fit(df.t, df.rec_volt)

# compute "impedence" objects
h_z_prim_sec = h_v_sec / h_i_prim
h_z_prim_rec = h_v_rec / h_i_prim
h_z_sec_rec = h_v_rec / h_v_sec



In [ ]:
%%opts RGB [width=800, height=350]
x, y = df.t, df.sec_volt
yf = h_v_sec.predict(df.t)

((
    shade(hv.Curve((x, y)), color='blue')
    *shade(hv.Curve((x, yf)), color='red')
) + (
    shade(hv.Curve((x, y - yf), vdims=['sss']), color='blue')
)).cols(1)


In [ ]:
%%opts RGB [width=800, height=350]
x, y = df.t, df.sec_volt
yf = h_v_sec.predict(df.t)

((
    shade(hv.Curve((x, y)), color='blue')
    *shade(hv.Curve((x, yf)), color='red')
) + (
    shade(hv.Curve((x, y - yf), vdims=['sss']), color='blue')
)).cols(1)


In [ ]:
%%opts RGB [width=800, height=350]
from scipy import signal
t, y = df.t.values, df.sec_volt.values

b, a = signal.butter(8, 0.01)
yf1 = signal.filtfilt(b, a, y, padlen=150)

kd = hv.Dimension('time', range=(0, .1))
vd = hv.Dimension('amp', range=(-6.5, 6.5))

(
shade(hv.Curve((t, y), kdims=[kd], vdims=[vd]))    
*shade(hv.Curve((t, yf1), kdims=[kd], vdims=[vd]), color='red')  
*shade(hv.Curve((t, yf2), kdims=[kd], vdims=[vd]), color='green')  
    
)


In [ ]:
.005 * round(.5 / (t[1] - t[0]))

In [ ]:
class SFFT:
    def _get_padded_length(self, initial_length, interp_exp=0):
        for nn in range(int(1e6)):
            padded_length = 2 ** nn
            if padded_length >= initial_length:
                break
        return padded_length * 2 ** interp_exp

    def fft(self, time, amplitude, interp_exp=3):
        # demean the signal
        amplitude = amplitude - np.mean(amplitude)

        # pad length to power of two with maybe some interpolation
        padded_length = self._get_padded_length(len(amplitude), interp_exp=interp_exp)

        # get the sample time
        dt = np.median(np.diff(time))

        # compute the fft
        z = np.fft.fft(amplitude, n=padded_length)

        # define a slice for postive frequencies
        ind = slice(0, int((len(z) / 4)))

        # get positive amplitudes
        amp_f = np.abs(z)[ind]

        # compute positive freqs
        f = np.fft.fftfreq(len(z), d=dt)[ind]
        return f, amp_f
   
f, amp = SFFT().fft(df.t, df.sec_volt - df.sec_volt.mean())

In [ ]:
%%opts Curve [width=800, height=350 logy=False, logx=False]
hv.Curve((f, amp))

In [ ]:
pl.loglog(f, amp)

In [ ]:
from astropy import units as u

In [ ]:
C = 80 * u.uF
(1. / (1j * 2 * np.pi * 50000 * u.Hz * C)).to(u.Ohm)

In [ ]:
f = 50 * u.kHz
w = 2 * np.pi * f
C = 10 * u.uF
X = 1. / (1j * w * C)
X.to(u.Ohm)

In [ ]:
# demean the signal
amplitude = amplitude - np.mean(amplitude)

# pad length to power of two with maybe some interpolation
padded_length = self._get_padded_length(len(amplitude), interp_exp=interp_exp)

# get the sample time
dt = np.median(np.diff(time))

# compute the fft
z = fft(amplitude, n=padded_length)

# define a slice for postive frequencies
ind = slice(0, int((len(z) / 2)))

# get positive amplitudes
amp_f = np.abs(z)[ind]

# compute positive freqs
f = np.fft.fftfreq(len(z), d=dt)[ind]

# return the max freq
return f[np.where(amp_f == np.max(amp_f))[0]][0]